# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
import json
import os
import sys
import glob
from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
import numpy
import pickle
import matplotlib.pyplot as plt

import jax
from jax import jit
from noisyopt import minimizeSPSA, minimizeCompass

from discopy.quantum import Circuit
from discopy.tensor import Tensor
from discopy.utils import loads
#from pytket.extensions.qiskit import AerBackend
#from pytket.extensions.qulacs import QulacsBackend
#from pytket.extensions.cirq import CirqStateSampleBackend
backend = None

from utils import get_symbols, construct_data_and_labels, select_circuits, read_diagrams, create_labeled_classes, bin_class_loss, multi_class_loss, bin_class_acc, multi_class_acc, visualize_result_noisyopt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings('ignore')
this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

EPOCHS = 500
SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

rng = numpy.random.default_rng(SEED)
numpy.random.seed(SEED)

## Read circuit data

We read the circuits from the pickled files. Select if we perform binary classification or multi-class classification. Give number of qubits to create classes:
- 1 qubits -> 2^1 = 2 classes i.e. binary classification
- 2 qubits -> 2^2 = 4 classes
- ...
- 5 qubits -> 2^5 = 32 classes, etc.

In [2]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 1
layers = 1
single_qubit_params = 3
n_wire_count = 1

loss = multi_class_loss
acc = multi_class_acc

if classification == 1:
    loss = bin_class_loss
    acc = bin_class_acc

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [3]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [4]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Lambeq optimizer

## Model

In [5]:
def make_pred_fn(circuits):
    # In the case we want to use other backends. 
    # Currently does not work properly.
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

## Loss function and evaluation

In [6]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)

        cost = loss(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Minimization with noisyopt

In [7]:
def initialize_parameters(old_params, old_values, new_params):
    new_values = list(numpy.array(rng.random(len(new_params))))
    old_param_dict = {}
    for p, v in zip(old_params, old_values):
        old_param_dict[p] = v
        
    parameters = sorted(set(old_params + new_params), key=default_sort_key)
    values = []
    for p in parameters:
        if p in old_param_dict:
            values.append(old_param_dict[p])
        else:
            values.append(new_values.pop())
            
    return parameters, np.array(values)

In [ ]:
EPOCHS = 3000
syms = {}
limit = False
all_training_keys = list(training_circuits.keys())
initial_circuit_keys = all_training_keys[:5]
current_training_circuits = {}
result_file = workload_size + "_noisyopt_2" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

for k in initial_circuit_keys:
    current_training_circuits[k] = training_circuits[k]
    
syms = get_symbols(current_training_circuits)
parameters = sorted(syms, key=default_sort_key)
initial_number_of_circuits = 44
if initial_number_of_circuits > 5 and os.path.exists("points//" + result_file + ".npz"):
    with open("points//" + result_file + ".npz", "rb") as f:
        print("Loading parameters from file " + result_file)
        npzfile = np.load(f)
        init_params_spsa = npzfile['arr_0']
else:
    print("Initializing new parameters")
    init_params_spsa = np.array(rng.random(len(parameters)))
result = None
run = 0

In [ ]:
for i, key in enumerate(all_training_keys[initial_number_of_circuits:]):
    print("Progress: ", round((i + initial_number_of_circuits)/len(all_training_keys), 3))
    
    if len(syms) == len(get_symbols(current_training_circuits)) and i > 0:
        if i != len(all_training_keys[1:]):
            current_training_circuits[key] = training_circuits[key]
            new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
            if result:
                parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)
                #continue
            else:
                syms = get_symbols(current_training_circuits)
                parameters = sorted(syms, key=default_sort_key)
                init_params_spsa = np.array(rng.random(len(parameters)))
    else:
        run += 1
    
    # Select those circuits from test and validation circuits which share the parameters with the current training circuits
    current_validation_circuits = select_circuits(current_training_circuits, validation_circuits)
    current_test_circuits = select_circuits(current_training_circuits, test_circuits)
    
    if len(current_validation_circuits) == 0 or len(current_test_circuits) == 0:
        continue
    
    # Create lists with circuits and their corresponding label
    training_circuits_l, training_data_labels_l = construct_data_and_labels(current_training_circuits, training_data_labels)
    validation_circuits_l, validation_data_labels_l = construct_data_and_labels(current_validation_circuits, validation_data_labels)
    test_circuits_l, test_data_labels_l = construct_data_and_labels(current_test_circuits, test_data_labels)
    
    # Limit the number of validation and test circuits to 20% of number of the training circuits
    if limit:
        val_test_circ_size = ceil(len(current_training_circuits))
        if len(current_validation_circuits) > val_test_circ_size:
            validation_circuits_l = validation_circuits_l[:val_test_circ_size]
            validation_data_labels_l = validation_data_labels_l[:val_test_circ_size]
        if len(current_test_circuits) > val_test_circ_size:
            test_circuits_l = test_circuits_l[:val_test_circ_size]
            test_data_labels_l = test_data_labels_l[:val_test_circ_size]
    
    stats = f"Number of training circuits: {len(training_circuits_l)}   "\
        + f"Number of validation circuits: {len(validation_circuits_l)}   "\
        + f"Number of test circuits: {len(test_circuits_l)}   "\
        + f"Number of parameters in model: {len(set([sym for circuit in training_circuits_l for sym in circuit.free_symbols]))}"
    
    with open("results//" + result_file + ".txt", "a") as f:
        f.write(stats + "\n")
    
    print(stats)
    
    train_pred_fn = jit(make_pred_fn(training_circuits_l))
    dev_pred_fn = jit(make_pred_fn(validation_circuits_l))
    test_pred_fn = make_pred_fn(test_circuits_l)
    
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)
    
    def callback_fn(xk):
        #print(xk)
        valid_loss = dev_cost_fn(xk)
        train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
        train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
        valid_acc = numpy.around(float(dev_accs[-1]), 4)
        iters = int(len(train_accs)/2)
        if iters % 200 == 0:
            info = f"Epoch: {iters}   "\
            + f"train/loss: {train_loss}   "\
            + f"valid/loss: {numpy.around(float(valid_loss), 4)}   "\
            + f"train/acc: {train_acc}   "\
            + f"valid/acc: {valid_acc}"
        
            with open("results//" + result_file + ".txt", "a") as f:
                f.write(info + "\n")
                
            print(info, file=sys.stderr)
        return valid_loss
    
    a_value = 0.0053
    c_value = 0.0185
            
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)

    result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a = a_value, c = c_value, niter=EPOCHS, callback=callback_fn)
    #result = minimizeCompass(train_cost_fn, x0=init_params_spsa, redfactor=2.0, deltainit=1.0, deltatol=0.001, feps=1e-15, errorcontrol=True, funcNinit=30, funcmultfactor=2.0, paired=True, alpha=0.05, callback=callback_fn)

    figure_path = this_folder + "//results//" + result_file + ".png"
    visualize_result_noisyopt(result, make_cost_fn, test_pred_fn, test_data_labels_l, train_costs, train_accs, dev_costs, dev_accs, figure_path, result_file)
    
    run += 1
    EPOCHS += 100
    syms = get_symbols(current_training_circuits)
    
    # Extend for the next optimization round
    current_training_circuits[key] = training_circuits[key]
    new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
    parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)

Progress:  0.022
Number of training circuits: 5   Number of validation circuits: 5   Number of test circuits: 8   Number of parameters in model: 107


Epoch: 200   train/loss: 0.2508   valid/loss: 0.9986   train/acc: 1.0   valid/acc: 0.6
Epoch: 400   train/loss: 0.1625   valid/loss: 0.9911   train/acc: 1.0   valid/acc: 0.6
Epoch: 600   train/loss: 0.1644   valid/loss: 0.9531   train/acc: 1.0   valid/acc: 0.6
Epoch: 800   train/loss: 0.1481   valid/loss: 0.8764   train/acc: 1.0   valid/acc: 0.6
Epoch: 1000   train/loss: 0.1283   valid/loss: 0.8315   train/acc: 1.0   valid/acc: 0.8
Epoch: 1200   train/loss: 0.1125   valid/loss: 0.782   train/acc: 1.0   valid/acc: 0.8
Epoch: 1400   train/loss: 0.1229   valid/loss: 0.828   train/acc: 1.0   valid/acc: 0.6
Epoch: 1600   train/loss: 0.1333   valid/loss: 0.812   train/acc: 1.0   valid/acc: 0.8
Epoch: 1800   train/loss: 0.1177   valid/loss: 0.7483   train/acc: 1.0   valid/acc: 0.8
Epoch: 2000   train/loss: 0.0992   valid/loss: 0.7026   train/acc: 1.0   valid/acc: 0.8
Epoch: 2200   train/loss: 0.1027   valid/loss: 0.7244   train/acc: 1.0   valid/acc: 0.8
Epoch: 2400   train/loss: 0.1009   vali

Test accuracy: 0.875
Progress:  0.025
Number of training circuits: 6   Number of validation circuits: 18   Number of test circuits: 19   Number of parameters in model: 134


Epoch: 200   train/loss: 0.1072   valid/loss: 0.9479   train/acc: 1.0   valid/acc: 0.5556
Epoch: 400   train/loss: 0.108   valid/loss: 1.0108   train/acc: 1.0   valid/acc: 0.5556
Epoch: 600   train/loss: 0.0829   valid/loss: 1.0398   train/acc: 1.0   valid/acc: 0.5556
Epoch: 800   train/loss: 0.078   valid/loss: 1.0985   train/acc: 1.0   valid/acc: 0.5556
Epoch: 1000   train/loss: 0.0708   valid/loss: 1.1792   train/acc: 1.0   valid/acc: 0.5556
Epoch: 1200   train/loss: 0.0526   valid/loss: 1.2111   train/acc: 1.0   valid/acc: 0.5556
Epoch: 1400   train/loss: 0.0541   valid/loss: 1.2503   train/acc: 1.0   valid/acc: 0.5556
Epoch: 1600   train/loss: 0.076   valid/loss: 1.4186   train/acc: 1.0   valid/acc: 0.5556
Epoch: 1800   train/loss: 0.0533   valid/loss: 1.4742   train/acc: 1.0   valid/acc: 0.5556
Epoch: 2000   train/loss: 0.0951   valid/loss: 1.4268   train/acc: 1.0   valid/acc: 0.5556
Epoch: 2200   train/loss: 0.0685   valid/loss: 1.4183   train/acc: 1.0   valid/acc: 0.5556
Epoch:

Test accuracy: 0.6315789
Progress:  0.027
Number of training circuits: 7   Number of validation circuits: 22   Number of test circuits: 25   Number of parameters in model: 137


...

Test accuracy: 0.7173913
Progress:  0.087
Number of training circuits: 34   Number of validation circuits: 96   Number of test circuits: 92   Number of parameters in model: 223


Epoch: 200   train/loss: 0.1723   valid/loss: 0.9181   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 400   train/loss: 0.1769   valid/loss: 0.9151   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 600   train/loss: 0.1491   valid/loss: 0.9181   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 800   train/loss: 0.1455   valid/loss: 0.9209   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 1000   train/loss: 0.1807   valid/loss: 0.9232   train/acc: 0.9412   valid/acc: 0.5312
Epoch: 1200   train/loss: 0.1675   valid/loss: 0.9231   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 1400   train/loss: 0.1836   valid/loss: 0.9233   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 1600   train/loss: 0.1667   valid/loss: 0.924   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 1800   train/loss: 0.1519   valid/loss: 0.9198   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 2000   train/loss: 0.1676   valid/loss: 0.9235   train/acc: 0.9706   valid/acc: 0.5312
Epoch: 2200   train/loss: 0.1342   valid/loss: 0.9201   train/acc

Test accuracy: 0.7282609
Progress:  0.089
Number of training circuits: 35   Number of validation circuits: 97   Number of test circuits: 96   Number of parameters in model: 229


Epoch: 200   train/loss: 0.2036   valid/loss: 0.8974   train/acc: 0.9143   valid/acc: 0.5464
Epoch: 400   train/loss: 0.1706   valid/loss: 0.8931   train/acc: 0.9714   valid/acc: 0.5361
Epoch: 600   train/loss: 0.1537   valid/loss: 0.9221   train/acc: 0.9429   valid/acc: 0.5361
Epoch: 800   train/loss: 0.1652   valid/loss: 0.9176   train/acc: 0.9714   valid/acc: 0.5361
Epoch: 1000   train/loss: 0.1631   valid/loss: 0.9232   train/acc: 0.9714   valid/acc: 0.5258
Epoch: 1200   train/loss: 0.1253   valid/loss: 0.9389   train/acc: 0.9714   valid/acc: 0.5258
Epoch: 1400   train/loss: 0.1467   valid/loss: 0.9393   train/acc: 0.9714   valid/acc: 0.5361
Epoch: 1600   train/loss: 0.1356   valid/loss: 0.9466   train/acc: 0.9714   valid/acc: 0.5361
Epoch: 1800   train/loss: 0.1403   valid/loss: 0.9532   train/acc: 0.9714   valid/acc: 0.5361
Epoch: 2000   train/loss: 0.1444   valid/loss: 0.9481   train/acc: 0.9714   valid/acc: 0.5361
Epoch: 2200   train/loss: 0.1347   valid/loss: 0.9528   train/ac

Test accuracy: 0.7291667
Progress:  0.092
Number of training circuits: 36   Number of validation circuits: 97   Number of test circuits: 96   Number of parameters in model: 229
